In [15]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score
import datasets
import sentence_transformers
from sentence_transformers import SentenceTransformer


In [6]:
train=datasets.load_from_disk('D:\\SBERT-Training\\datasets\\ContractNLI\\train')
train

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 6819
})

In [10]:
test=datasets.load_from_disk('D:\\SBERT-Training\\datasets\\ContractNLI\\test')
test

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 1991
})

In [11]:

# Load the dataset
train_df=pd.DataFrame({'premise':train['premise'],
                         'hypothesis':train['hypothesis'],
                         'label':train['label']})
train_df


,premise,hypothesis,label
0,2.3 Provided that the Recipient has a written ...,Receiving Party shall not reverse engineer any...,2
1,5. All Confidential Information in any form an...,Receiving Party shall destroy or return some C...,1
2,4. Nothing in this Agreement is to be construe...,Agreement shall not grant Receiving Party any ...,1
3,11. The Recipient shall not advertise or other...,Receiving Party shall not disclose the fact th...,1
4,"1. “Confidential Information”, whenever used i...",Confidential Information shall only include te...,2
...,...,...,...
6814,"For purposes of this Agreement, the Provider’s...",Confidential Information may include verbally ...,1
6815,"“Confidential Information” does not include, a...",Receiving Party may create a copy of some Conf...,2
6816,(c) If the Recipient or any of the Recipient’s...,Receiving Party shall notify Disclosing Party ...,1
6817,"However, the Provider’s “Confidential Informat...",Receiving Party may acquire information simila...,1


In [12]:
test_df=pd.DataFrame({'premise':test['premise'],
                         'hypothesis':test['hypothesis'],
                         'label':test['label']})
test_df



,premise,hypothesis,label
0,In the event that a receiving party or its Ass...,Receiving Party shall not reverse engineer any...,2
1,The Recipient shall immediately return and red...,Receiving Party shall destroy or return some C...,1
2,"No other right or license, whether expressed o...",Agreement shall not grant Receiving Party any ...,1
3,In the event that a receiving party or its Ass...,Receiving Party shall not disclose the fact th...,2
4,"For purposes of this Agreement, ""Confidential ...",Confidential Information shall only include te...,0
...,...,...,...
1986,"2. As used herein, “Confidential Information” ...",Receiving Party shall not solicit some of Disc...,2
1987,9. Upon the Disclosing Party’s written request...,Receiving Party may create a copy of some Conf...,2
1988,The Recipient agrees that if the Recipient is ...,Receiving Party shall notify Disclosing Party ...,1
1989,1.2 Notwithstanding anything herein to the con...,Receiving Party may acquire information simila...,1


In [16]:
path='D:\\distilbert-base-uncased'
model=SentenceTransformer(path)

No sentence-transformers model found with name D:\distilbert-base-uncased. Creating a new one with mean pooling.


In [18]:

# Load the pre-trained model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\aakha\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aakha\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
type(train_df['premise'])

pandas.core.series.Series

In [22]:

# Preprocess the data
train_encodings = tokenizer(train_df['premise'].to_list(), train_df['hypothesis'].to_list(), 
                             return_tensors='pt', 
                             max_length=512, 
                             padding='max_length', 
                             truncation=True)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [23]:

test_encodings = tokenizer(test_df['premise'].to_list(), test_df['hypothesis'].to_list(), 
                            return_tensors='pt', 
                            max_length=512, 
                            padding='max_length', 
                            truncation=True)


In [24]:

# Create labels
train_labels = train_df['label']
test_labels = test_df['label']


In [26]:
import torch

In [32]:

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(**train_encodings)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)
    train_accuracy = accuracy_score(train_labels, predictions.cpu().numpy())

    outputs = model(**test_encodings)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)
    test_accuracy = accuracy_score(test_labels, predictions.cpu().numpy())


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 10725359616 bytes.

In [31]:

print("Multinli Train Accuracy:", train_accuracy)
print("Multinli Test Accuracy:", test_accuracy)

NameError: name 'train_accuracy' is not defined